In [ ]:
RDATEC = 202002 #/*current month*/
RDATEP = 202001 #/*last month*/

RATE_1 = '29/02/2020'  #currentdate
RATE_2 = '31/01/2020'  #lastdate
R_YEAR = '02/2020'     #currentdate

import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan

#/******************************* Home Financing (HOME_RDATE) *******************************/

HF_HOME = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\05_Home_Financing\04_Dataset\\"+str(RDATEC)+"\mortgage_"+str(RDATEC)+".txt", sep = "\t", header = 0,encoding='cp1252',low_memory=False)
DSR_TL = pd.read_excel(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\18_FRAN_DSR\\"+str(RDATEC)+"\DSR_TL_"+str(RDATEC)+".xlsx")
DSR_OD = pd.read_excel(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\18_FRAN_DSR\\"+str(RDATEC)+"\DSR_OD_"+str(RDATEC)+".xlsx")

HF_HOME.columns = HF_HOME.columns.str.replace(" ", "_").str.replace(".", "_")
DSR_TL.columns = DSR_TL.columns.str.replace(" ", "_")
DSR_OD.columns = DSR_OD.columns.str.replace(" ", "_")



In [6]:
HF_HOME_P = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\05_Home_Financing\04_Dataset\\"+str(RDATEP)+"\mortgage_"+str(RDATEP)+".txt", sep = "\t", header = 0,encoding='cp1252',low_memory=False)
DSR_TL_P = pd.read_excel(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\18_FRAN_DSR\\"+str(RDATEP)+"\DSR_TL_"+str(RDATEP)+".xlsx")
DSR_OD_P = pd.read_excel(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\18_FRAN_DSR\\"+str(RDATEP)+"\DSR_OD_"+str(RDATEP)+".xlsx")

HF_HOME_P.columns = HF_HOME_P.columns.str.replace(" ", "_").str.replace(".", "_")
DSR_TL_P.columns = DSR_TL_P.columns.str.replace(" ", "_")
DSR_OD_P.columns = DSR_OD_P.columns.str.replace(" ", "_")

In [7]:


#====================================HF=================================================================

DSR_TL1 = DSR_TL.groupby(['M_Account_No'])[['Dsr_Ratio']].max().reset_index()
DSR_OD1 = DSR_OD.groupby(['Account_No'])[['Dsr_Ratio']].max().reset_index()

DSR_OD1.Dsr_Ratio.fillna(-9999, inplace=True)
DSR_OD1.Dsr_Ratio = DSR_OD1.Dsr_Ratio.map(float)

DSR_TL1.Dsr_Ratio.fillna(-9999, inplace=True)
DSR_TL1.Dsr_Ratio = DSR_TL1.Dsr_Ratio.map(float)

DSR_OD1.Account_No = DSR_OD1.Account_No.map(str)
DSR_TL1.M_Account_No = DSR_TL1.M_Account_No.map(str)
HF_HOME.M_ACCOUNT_NO = HF_HOME.M_ACCOUNT_NO.map(str)

HOME = HF_HOME.merge(DSR_OD1.rename(columns = {'Account_No':'M_ACCOUNT_NO','Dsr_Ratio':'Dsr_Ratio2'}), on="M_ACCOUNT_NO", how = 'left', indicator = 'exists').merge(DSR_TL1.rename(columns = {'M_Account_No':'M_ACCOUNT_NO','Dsr_Ratio':'Dsr_Ratio1'}), on="M_ACCOUNT_NO", how = 'left', indicator = True)

def DSR_RAT(_merge,Dsr_Ratio1,exists,Dsr_Ratio2):
    if _merge == 'both':
        return Dsr_Ratio1
    elif exists == 'both':
        return Dsr_Ratio2
HOME['DSR_RAT'] = HOME.apply(lambda x: DSR_RAT(x['_merge'], x['Dsr_Ratio1'],x['exists'],x['Dsr_Ratio2']), axis=1)

HOME2 = HOME.drop(['exists','_merge'], axis=1)

import datetime
RDATE1 = pd.to_datetime(RATE_1, format='%d/%m/%Y')
HOME2.loc[HOME2['M_DATE_FIRST_RELEASED'] == '?', "M_DATE_FIRST_RELEASED"] = RATE_1
HOME2.loc[HOME2['M_DATE_FIRST_RELEASED'] == 0, "M_DATE_FIRST_RELEASED"] = RATE_1
HOME2['X'] = RDATE1 - (pd.to_datetime(HOME2['M_DATE_FIRST_RELEASED'], format='%d/%m/%Y')) 
HOME2['MOB'] = (pd.to_numeric(HOME2['X'].dt.days, downcast='integer')/30.417) #30.417 tu hari dlm sebulan
HOME2.drop('X', axis=1, inplace=True)

#====================================HF_P=================================================================

DSR_TL_P1 = DSR_TL_P.groupby(['M_Account_No'])[['Dsr_Ratio']].max().reset_index()
DSR_OD_P1 = DSR_OD_P.groupby(['Account_No'])[['Dsr_Ratio']].max().reset_index()

DSR_OD_P1.Dsr_Ratio.fillna(-9999, inplace=True)
DSR_OD_P1.Dsr_Ratio = DSR_OD_P1.Dsr_Ratio.map(float)

DSR_TL_P1.Dsr_Ratio.fillna(-9999, inplace=True)
DSR_TL_P1.Dsr_Ratio = DSR_TL_P1.Dsr_Ratio.map(float)

DSR_OD_P1.Account_No = DSR_OD_P1.Account_No.map(str)
DSR_TL_P1.M_Account_No = DSR_TL_P1.M_Account_No.map(str)
HF_HOME_P.M_ACCOUNT_NO = HF_HOME_P.M_ACCOUNT_NO.map(str)

HOME_P = HF_HOME_P.merge(DSR_OD_P1.rename(columns = {'Account_No':'M_ACCOUNT_NO','Dsr_Ratio':'Dsr_Ratio2'}), on="M_ACCOUNT_NO", how = 'left', indicator = 'exists').merge(DSR_TL_P1.rename(columns = {'M_Account_No':'M_ACCOUNT_NO','Dsr_Ratio':'Dsr_Ratio1'}), on="M_ACCOUNT_NO", how = 'left', indicator = True)

HOME_P2 = HOME_P.drop(['exists','_merge'], axis=1)


import datetime
RDATE_P1 = pd.to_datetime(RATE_2, format='%d/%m/%Y')
HOME_P2.loc[HOME_P2['M_DATE_FIRST_RELEASED'] == '?', "M_DATE_FIRST_RELEASED"] = RATE_2
HOME_P2.loc[HOME_P2['M_DATE_FIRST_RELEASED'] == 0, "M_DATE_FIRST_RELEASED"] = RATE_2
HOME_P2['X'] = RDATE_P1 - (pd.to_datetime(HOME_P2['M_DATE_FIRST_RELEASED'], format='%d/%m/%Y')) 
HOME_P2['MOB'] = (pd.to_numeric(HOME_P2['X'].dt.days, downcast='integer')/30.417) #30.417 tu hari dlm sebulan
HOME_P2.drop('X', axis=1, inplace=True)

#-------------------------------------------HF001-------------------------------------------

HF001 = HOME2.iloc[np.where((HOME2.FIN_TYPE == 'I')&(HOME2.LEVEL_6 == 'Housing'))]

HF001.rename(columns = {'RISK_GRADE_VAL':'RISK_GRADE_VAL1',\
                        'M_NPL_DATE':'M_NPL_DATE1',\
                        'GCIF_INCOME':'GCIF_INCOME1',\
                        'OCCUPATION_SECTOR':'OCCUPATION_SECTOR1',\
                        'GENDER':'GENDER1',\
                        'RACE':'RACE1',\
                        'AGE_RANGE':'AGE_RANGE1',\
                        'PROP_DESC':'PROP_DESC1',\
                        'P_USAGE':'P_USAGE1',\
                        'AMRT_TERM':'AMRT_TERM1',\
                        'M_DATE_AA_APPROVED':'M_DATE_AA_APPROVED1',\
                        'PUR_PRC':'PUR_PRC1',\
                        'SNP':'SNP1',\
                        'CMV':'CMV1',\
                        'STATE':'STATE1',\
                        'CUSTOMER_SEGMENT':'CUSTOMER_SEGMENT1',\
                        'DSR_RAT':'DSR_RAT1'}, inplace = True)


def RATING(x):
    if x=='R1':
        return '01. R1'
    elif x=='R2':
        return '02. R2'
    elif x=='R3':
        return '03. R3'
    elif x=='R4':
        return '04. R4'
    elif x=='R5':
        return '05. R5'
    elif x=='R6':
        return '06. R6'
    elif x=='R7':
        return '07. R7'
    elif x=='R8':
        return '08. R8'
    elif x=='R9':
        return '09. R9'
    elif x=='R10':
        return '10. R10'
    elif x=='R11':
        return '11. R11'
    else: 
        return 'Unrated'
HF001['RISK_GRADE_VAL'] = HF001['RISK_GRADE_VAL1'].apply(RATING)

HF001.loc[HF001.M_NPL_FLG=='Y','RISK_GRADE_VAL']='IF'

def INCOM(x):
    if x=='Up to RM999':
        return '01. Up to RM3,000'
    elif x=='RM1,000 to RM1,999':
        return '01. Up to RM3,000'
    elif x=='RM2,000 to RM2,999':
        return '01. Up to RM3,000'
    elif x=='RM3,000 to RM3,999':
        return '02. Above RM3,000 to RM5,000'
    elif x=='RM4,000 to RM4,999':
        return '02. Above RM3,000 to RM5,000'
    elif x=='RM5,000 to RM5,999':
        return '03. Above RM5,000 to RM8,000'
    elif x=='RM6,000 to RM6,999':
        return '03. Above RM5,000 to RM8,000'
    elif x=='RM7,000 to RM7,999':
        return '03. Above RM5,000 to RM8,000'
    elif x=='RM8,000 to RM8,999':
        return '04. Above RM8,000 to RM15,000'
    elif x=='RM9,000 to RM9,999':
        return '04. Above RM8,000 to RM15,000'
    elif x=='RM10,000 to RM10,999':
        return '04. Above RM8,000 to RM15,000'
    elif x=='RM11,000 to RM14,999':
        return '04. Above RM8,000 to RM15,000'
    elif x=='RM15,000 to RM20,000':
        return '05. Above RM15,000 to RM20,000'
    elif x=='Above RM 20,000':
        return '06. Above RM20,000'
    else: 
        return 'Not Populated'
HF001['GCIF_INCOME'] = HF001['GCIF_INCOME1'].apply(INCOM)


HF001['OCCUPATION_SECTOR'] = HF001.OCCUPATION_SECTOR1
HF001.loc[HF001.OCCUPATION_SECTOR=='NOT APPLICABLE','OCCUPATION_SECTOR'] = 'ZA_OTHERS'
HF001.loc[HF001.OCCUPATION_SECTOR=='?','OCCUPATION_SECTOR'] = 'ZB_NOT POPULATED'

def GENDER(x):
    if x =='FEMALE':
        return 'Female'
    elif x == 'MALE':
        return 'Male'
    else:
        return 'Not Populated'
HF001['GENDER'] = HF001['GENDER1'].apply(GENDER)
    
def RACE(x):
    if x == 'CHINESE':
        return 'Chinese'
    elif x == 'INDIAN':
        return 'Indian'
    elif x == 'MALAY':
        return 'Malay'
    elif x == 'NATIVE':
        return 'Native'
    else:
        return 'Others'
HF001['RACE'] = HF001['RACE1'].apply(RACE)

def AGE_RANGE(x):
    if x == '<25 years':
        return '01. Up to 25 Years Old'
    elif x == '25 to 29  years':
        return '02. 25 to 29 Years Old'
    elif x == '30 to 34 years':
        return '03. 30 to 34 Years Old'
    elif x == '35 to 39  years':
        return '04. 35 to 39 Years Old'
    elif x == '40 to 44 years':
        return '05. 40 to 44 Years Old'
    elif x == '45 to 49 years':
        return '06. 45 to 49 Years Old'
    elif x == '50 to 54 years':
        return '07. 50 to 54 Years Old'
    elif x == '55 years and above':
        return '08. 55 Years & Above'
    else:
        return 'Not Populated'
HF001['RACE'] = HF001['RACE1'].apply(RACE)

def STATE(x):
    if x == 'LABUAN':
        return 'FED. TERRITORY'
    elif x == 'K.L MAIN':
        return 'FED. TERRITORY'
    else:
        return x
HF001['STATE'] = HF001['STATE1'].apply(STATE)

HF001['OSM'] = HF001.M_BNM_BALANCE/1000000   #/* OUTSTANDING IN MIL */
HF001['NOA'] = 1                             #/* NUMBER OF ACCOUNT  */

HF001.loc[HF001['M_MTHS_INSTALLMENT_IN_ARREARS'] == '?', "M_MTHS_INSTALLMENT_IN_ARREARS"] = -9999
HF001.M_MTHS_INSTALLMENT_IN_ARREARS.fillna(-9999, inplace=True)
HF001['M_MTHS_INSTALLMENT_IN_ARREARS'] = HF001['M_MTHS_INSTALLMENT_IN_ARREARS'].astype(int)

# /* 2 MIA */ 
HF001.loc[HF001.M_MTHS_INSTALLMENT_IN_ARREARS == 2,'OSM_MIA2'] = HF001.M_BNM_BALANCE/1000000
HF001.loc[HF001.M_MTHS_INSTALLMENT_IN_ARREARS == 2,'NOA_MIA2'] = 1

# /* 3 MIA */
HF001.loc[HF001.M_MTHS_INSTALLMENT_IN_ARREARS == 3,'OSM_MIA3'] = HF001.M_BNM_BALANCE/1000000
HF001.loc[HF001.M_MTHS_INSTALLMENT_IN_ARREARS == 3,'NOA_MIA3'] = 1

# /* IF */
HF001.loc[HF001.M_NPL_FLG == 'Y','OSM_IF'] = HF001.M_BNM_BALANCE/1000000
HF001.loc[HF001.M_NPL_FLG == 'Y','NOA_IF'] = 1

# /* NCIF */
HF001['R_YEAR1'] = HF001.M_NPL_DATE1.str[-7:]
HF001.loc[(HF001.M_NPL_FLG == 'Y')&(HF001.R_YEAR1 == R_YEAR),'OSM_NCIF'] = HF001.M_BNM_BALANCE/1000000 #&(HF001.M_NPL_DATE1 == RATE_1)
HF001.loc[(HF001.M_NPL_FLG == 'Y')&(HF001.R_YEAR1 == R_YEAR),'NOA_NCIF'] = 1                           #&(HF001.M_NPL_DATE1 == RATE_1)
HF001.drop('R_YEAR1', axis=1, inplace=True)

# /* TO DETERMINE WHICH VALUE TO BE TAKEN FOR CALCULATION OF MOF AND FTV */
HF001.loc[HF001.PUR_PRC1 == '?', "PUR_PRC1"] = 0
HF001.PUR_PRC1 = HF001.PUR_PRC1.map(float)
HF001['PUR_PRC'] = HF001.PUR_PRC1*1

# /*OMV_WOLOC=OMV_WOLOC1*1;*/
HF001['SNP'] = HF001.SNP1*1
HF001['CMV'] = HF001.CMV1*1

HF001.loc[HF001.SNP == '?', "SNP"] = 0
HF001.loc[HF001.CMV == '?', "CMV"] = 0
HF001.SNP = HF001.SNP.astype(float)
HF001.CMV = HF001.CMV.astype(float)

def PURCHASE_PRICE(PUR_PRC,SNP,CMV):
    if PUR_PRC>0:
        return PUR_PRC
    else:
        if SNP>0:
            return SNP
        else:
            return CMV
HF001['PURCHASE_PRICE'] = HF001.apply(lambda x: PURCHASE_PRICE(x['PUR_PRC'],x['SNP'],x['CMV']), axis=1)

# /* MARGIN OF FINANCE(MOF) & FINANCING TO VALUE(FTV) */
HF001['MOF1'] = HF001.M_APPROVED_LIMIT/HF001.PURCHASE_PRICE
HF001['FTV1'] = HF001.M_BNM_BALANCE/HF001.PURCHASE_PRICE

def PERCEN(x):
    if x == float('inf'):
        return 'Not Populated'
    elif x > 0 and x <= 0.3:
        return '01. Up to 30%'
    elif x > 0.3 and x <= 0.5:
        return '02. > 30% to 50%' 
    elif x > 0.5 and x <= 0.6:
        return '03. > 50% to 60%'
    elif x > 0.6 and x <= 0.7:
        return '04. > 60% to 70%'
    elif x > 0.7 and x <= 0.75:
        return '05. > 70% to 75%'
    elif x > 0.75 and x <= 0.8:
        return '06. > 75% to 80%'
    elif x > 0.8 and x <= 0.85:
        return '07. > 80% to 85%'
    elif x > 0.85 and x <= 0.9:
        return '08. > 85% to 90%'
    elif x > 0.9:
        return '09. >90%'
    else:
        return 'Not Populated'
HF001['MOF'] = HF001.MOF1.apply(PERCEN)
HF001['FTV'] = HF001.FTV1.apply(PERCEN)

# /* PROVERTY VALUE */  
def PURVAL(x):
    if x > 0 and x <= 25000:
        return '01. <=RM25k'
    elif x > 25000 and x <= 42000:
        return '02. <=RM42k'
    elif x > 42000 and x <= 60000:
        return '03. <=RM60k'
    elif x > 60001 and x <= 100000:
        return '04. <=RM100k'
    elif x > 100000 and x <= 150000:
        return '05. <=RM150k'
    elif x > 150000 and x <= 200000:
        return '06. <=RM200k'
    elif x > 200000 and x <= 250000:
        return '07. <=RM250k'
    elif x > 250000 and x <= 500000:
        return '08. <=RM500k'
    elif x > 500000 and x <= 1000000:
        return '09. <=RM1mil'
    elif x > 1000000:
        return '10. >RM1mil'
    else:
        return 'Not Populated'
HF001['PROPERTY_VALUE'] = HF001.PURCHASE_PRICE.apply(PERCEN)                

# /* TYPE OF RESIDENTIAL */
def TYPRES(x):
    if x == 'Terrace House':
        return '01. Terrace House'
    elif x == 'Semi-Detached House':
        return '02. Semi-Detached House'
    elif x == 'Detached House':
        return '03. Detached House'
    elif x == 'Flat/Apartment Unit':
        return '04. Flat/Apartment Unit'
    elif x == 'Condominium Unit':
        return '05. Condominium Unit'
    else:
        return 'Others'
HF001['PROP_DESC'] = HF001.PROP_DESC1.apply(TYPRES)
                
# /* PURPOSE */
def PURPO(x):
    if x == 'INV':
        return '01. Investment'
    elif x == 'OWN':
        return '02. Own'
    else:
        return 'Not Populated'
HF001['P_USAGE'] = HF001.P_USAGE1.apply(PURPO)
            
# /* FINANCING TENURE */
HF001.loc[HF001.AMRT_TERM1 == '?', "AMRT_TERM1"] = 99999
HF001.AMRT_TERM1 = HF001.AMRT_TERM1.astype(int)

def FTENRE(x):
    if x == 99999:
        return '.'
    elif x > 0 and x <= 5:
        return '01. Up to 5 Years'
    elif x > 5 and x <= 10:
        return '02. Up to 10 Years'
    elif x > 10 and x <= 15:
        return '03. Up to 15 Years'
    elif x > 15 and x <= 20:
        return '04. Up to 20 Years'
    elif x > 20 and x <= 25:
        return '05. Up to 25 Years'
    elif x > 25 and x <= 30:
        return '06. Up to 30 Years'
    elif x > 30:
        return '07. More than 30 Years';
HF001['AMRT_TERM'] = HF001.AMRT_TERM1.apply(FTENRE)

# /* YEAR APPROVED */
HF001['M_DATE_AA_APPROVED_TEMP'] = HF001.M_DATE_AA_APPROVED1.str[-4:]
HF001['M_DATE_AA_APPROVED_TEMP'] = HF001.M_DATE_AA_APPROVED_TEMP.astype(int)

def year(M_DATE_AA_APPROVED_TEMP):
    if M_DATE_AA_APPROVED_TEMP <= 2000:
        return '<=Year 2000'
    else:
        return 'Year '+str(M_DATE_AA_APPROVED_TEMP)
HF001['M_DATE_AA_APPROVED'] = HF001.M_DATE_AA_APPROVED_TEMP.apply(year)

#HF001['M_DATE_AA_APPROVED'] = 'Year ' + HF001.M_DATE_AA_APPROVED_TEMP

# /* LOAN SIZE */
def LOANSIZ(x):
    if x <= 150000:
        return '01. <=RM150K'
    elif x > 150000 and x <= 500000:
        return '02. RM150-500K'
    elif x > 500000 and x <= 1000000:
        return '03. RM500-1 mil'
    elif x > 1000000:
        return '>04. RM 1 mil'
HF001['LOAN_SIZE'] = HF001.M_APPROVED_LIMIT.apply(LOANSIZ)

# /* CUSTOMER SEGMENT */

def CUSTSEG(x):
    if x == 'HIGH NETWORTH':
        return '01. HNW'
    elif x == 'AFFLUENT':
        return '02. Affluent'
    elif x == 'EMERGING AFFLUENT':
        return '03. Emerging Affluent'
    elif x == 'MASS':
        return '04. Mass'
    else:
        return 'Not Populated'
HF001['CUSTOMER_SEGMENT'] = HF001.CUSTOMER_SEGMENT1.apply(CUSTSEG)

# /* DSR RATIO */
# /* DSR RATIO ADDED 6 JULY 2018 */



D:\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:334: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [8]:
HF001.DSR_RAT1 = HF001.DSR_RAT1.astype(float).fillna(0)

def DSRRATIO(x):
    if x >= 0.0001 and x <= 40:
        return 'DSR <=40%'
    elif x > 40 and x <= 60:
        return 'DSR >40% TO 60%'
    elif x > 60 and x <= 70:
        return 'DSR >60% TO 70%'
    elif x > 70 and x <= 80:
        return 'DSR >70% TO 80%'
    elif x > 80 and x <= 90:
        return 'DSR >80% TO 90%'
    elif x > 90:
        return 'DSR >90%'
    else:
        return 'NA'
HF001['DSR_RAT'] = HF001.DSR_RAT1.apply(DSRRATIO)

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:


#-------------------------------------------PMNTH_MIA--------------------------------------------

PMNTH_MIA = HOME_P2.iloc[np.where((HOME_P2.FIN_TYPE == 'I')&(HOME_P2.LEVEL_6 == 'Housing'))]

# /* PREVIOUS MONTH */
PMNTH_MIA.rename(columns={'M_MTHS_INSTALLMENT_IN_ARREARS':'M_MTHS_INSTALLMENT_IN_ARREARS_P','M_BNM_BALANCE':'M_BNM_BALANCE_P'}, inplace=True)
PMNTH_MIA.drop_duplicates()
PMNTH_MIA.sort_values("M_ACCOUNT_NO", inplace=True)

# /* CURRENT MONTH */
HF001.drop_duplicates()
HF002 = HF001.sort_values("M_ACCOUNT_NO")


# /*WAS IS*/
HF002['M_ACCOUNT_NO'] = HF002['M_ACCOUNT_NO'].str.strip()
HF002['M_ACCOUNT_NO'] = HF002['M_ACCOUNT_NO'].str.upper()
PMNTH_MIA['M_ACCOUNT_NO'] = PMNTH_MIA['M_ACCOUNT_NO'].str.strip()
PMNTH_MIA['M_ACCOUNT_NO'] = PMNTH_MIA['M_ACCOUNT_NO'].str.upper()

HF003 = HF002.merge(PMNTH_MIA[['M_ACCOUNT_NO','M_MTHS_INSTALLMENT_IN_ARREARS_P','M_BNM_BALANCE_P']], on="M_ACCOUNT_NO", how = 'left', indicator = True)

HF003.loc[HF003['M_MTHS_INSTALLMENT_IN_ARREARS'] == '?', "M_MTHS_INSTALLMENT_IN_ARREARS"] = -9999
HF003.loc[HF003['M_MTHS_INSTALLMENT_IN_ARREARS_P'] == '?', "M_MTHS_INSTALLMENT_IN_ARREARS_P"] = -9999
HF003.M_MTHS_INSTALLMENT_IN_ARREARS.fillna(-9999, inplace=True)
HF003.M_MTHS_INSTALLMENT_IN_ARREARS_P.fillna(-9999, inplace=True)
HF003['M_MTHS_INSTALLMENT_IN_ARREARS'] = HF003['M_MTHS_INSTALLMENT_IN_ARREARS'].map(int)
HF003['M_MTHS_INSTALLMENT_IN_ARREARS_P'] = HF003['M_MTHS_INSTALLMENT_IN_ARREARS_P'].map(int)

def WIFLG(_merge,M_MTHS_INSTALLMENT_IN_ARREARS,M_MTHS_INSTALLMENT_IN_ARREARS_P):
    if _merge == 'both':
        if M_MTHS_INSTALLMENT_IN_ARREARS > M_MTHS_INSTALLMENT_IN_ARREARS_P:
            return 'Forward Flow'
        elif M_MTHS_INSTALLMENT_IN_ARREARS == 0 and M_MTHS_INSTALLMENT_IN_ARREARS_P > M_MTHS_INSTALLMENT_IN_ARREARS:
            return 'Regular'
        elif M_MTHS_INSTALLMENT_IN_ARREARS > 0 and M_MTHS_INSTALLMENT_IN_ARREARS_P > M_MTHS_INSTALLMENT_IN_ARREARS:
            return 'Push Back'
        elif M_MTHS_INSTALLMENT_IN_ARREARS == M_MTHS_INSTALLMENT_IN_ARREARS_P:
            return 'Standardized'
    if _merge == 'left_only':
        return 'New Account'
HF003['WIFLG'] = HF003.apply(lambda x: WIFLG(x['_merge'],x['M_MTHS_INSTALLMENT_IN_ARREARS'],x['M_MTHS_INSTALLMENT_IN_ARREARS_P']), axis=1)
HF003_1 = HF003.drop(['_merge'], axis=1)



D:\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\anaconda3\lib\sit

In [10]:
FTV = HF003_1.iloc[np.where(HF003_1.FTV=='09. >90%')]

In [11]:
row = ['OSM','NOA','OSM_MIA2','NOA_MIA2','OSM_MIA3','NOA_MIA3','OSM_IF','NOA_IF','OSM_NCIF','NOA_NCIF']

# /* BY PROPERTY PROFILE */
FTV_sum = HF003_1.groupby(['FTV'])[row].sum().reset_index()
FTV_sum

,FTV,OSM,NOA,OSM_MIA2,NOA_MIA2,OSM_MIA3,NOA_MIA3,OSM_IF,NOA_IF,OSM_NCIF,NOA_NCIF
0,01. Up to 30%,1607.84,20016,15.08,251.0,6.42,131.0,14.82,164.0,0.36,13.0
1,02. > 30% to 50%,3450.95,19028,38.45,270.0,10.81,92.0,17.38,103.0,1.95,10.0
2,03. > 50% to 60%,3140.47,13281,27.09,194.0,11.92,92.0,15.66,84.0,3.39,15.0
3,04. > 60% to 70%,6551.66,22721,61.93,287.0,17.59,119.0,22.52,88.0,2.07,16.0
4,05. > 70% to 75%,2908.37,10548,31.16,173.0,9.39,65.0,10.29,51.0,0.66,4.0
5,06. > 75% to 80%,3244.27,11964,38.25,172.0,14.95,86.0,28.23,83.0,1.22,7.0
6,07. > 80% to 85%,5070.36,16237,52.71,245.0,14.87,91.0,17.64,66.0,1.32,6.0
7,08. > 85% to 90%,10503.30,30183,88.00,322.0,27.42,108.0,34.92,82.0,2.75,14.0
8,09. >90%,12407.62,39511,155.01,467.0,42.98,172.0,133.61,371.0,8.91,33.0
9,Not Populated,559.25,11482,31.74,822.0,18.72,494.0,26.58,519.0,0.86,40.0


In [12]:
#-------------------------------------------SORTBY(EXCEL)

writer2 = pd.ExcelWriter(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\03_Adhoc_Reporting\2022\202209_FTV 90% Home Financing\FTV_90%_'+str(RDATEC)+'(python).xlsx',engine='xlsxwriter')


#FTV_sum.to_excel(writer2, sheet_name='FTV_summary', index = False)
FTV.to_excel(writer2, sheet_name='FTV_90%_base', index = False)


writer2.save()